Step 0: Imports and coefficients

In [1]:
import numpy as np
from numpy import random
from scipy.optimize import minimize
import cmath
from sklearn.preprocessing import StandardScaler

In [76]:
noise_power_density = 5.0119e-21
p_0 = -0.001
chan_realisations = 100 # number of different channel realisations for one channel
W = 10e6 # bandwidth (unit:Hz)
circuit_power = 1 # constant circuit power of transmitter (unit: Watt)
max_power = 1 # max transmission power (unit:Watt)
num_samples = 5000 # no of rows in dataset

# Define realistic bounds for an urban environment (1 km³ area)
x_range = (0, 1000)  # meters
y_range = (0, 1000)  # meters
z_range = (0, 50)    # meters, considering building heights

Step 1: Generate nodes in 3D space

In [3]:
def generate_random_coordinates(n_devices, x_range, y_range, z_range):
    """Generate random coordinates for devices in a 3D space.

    Args:
        n_devices (int): Number of devices to generate coordinates for.
        x_range (tuple): The range (min, max) for the x-coordinate.
        y_range (tuple): The range (min, max) for the y-coordinate.
        z_range (tuple): The range (min, max) for the z-coordinate.

    Returns:
        list: A list of tuples containing the (x, y, z) coordinates.
    """
    coordinates = np.zeros((n_devices, 3))
    for i in range(n_devices):
        x = np.random.uniform(*x_range)
        y = np.random.uniform(*y_range)
        z = np.random.uniform(*z_range)
        coordinates[i] = [x, y, z]

    return coordinates


Step 2: Define channel

In [4]:
def calculate_distance(tx, rx):
  """Calculate the Euclidean distance between two 3D coordinates.

    Args:
        tx: 3D coordinate of the transmitter (x, y, z)
        rx: 3D coordinate of the receiver (x, y, z)

    Returns:
        Distance between the two points.
    """
  return np.linalg.norm(np.array(tx) - np.array(rx))

In [55]:
def generate_channel_model(p_0, tx, rx):
  """Generate channel model between one tx and its rx

    Args:
        p_0: reference path loss at 1m
        tx: transmitter coordinates
        rx: list of receiver coordinates

    Returns:
        h: channel model(1 x len(rx) x chan_realisations vector with multiple channel realisations across all tx-rx channel pairs)
    """
  h = np.empty((1, len(rx), chan_realisations), dtype=complex)

  # tx = np.array(tx)
  # rx = np.array(rx)

  # # Reshape tx for broadcasting
  # tx_reshaped = tx[np.newaxis, :]  # Shape (1, 2)

  # # Calculate distances between tx and all receivers
  # distances = np.linalg.norm(rx - tx_reshaped, axis=1)  # Shape will be (number of receivers,)

  # # Add a small epsilon to avoid division by zero
  # epsilon = 1e-10
  # distances = np.maximum(distances, epsilon)

  # # Calculate large-scale fading component for all receivers
  # total_large_scale_fading = np.sqrt(p_0 / (distances ** 3))

  # # Generate small-scale fading using Rayleigh distribution for all channel realizations and receivers
  # small_scale_fading = np.random.rayleigh(scale=1.0, size=(1, len(rx), chan_realisations))

  # # Compute the channel model (broadcast large-scale fading to match small-scale fading shape)
  # h = total_large_scale_fading[np.newaxis, :, np.newaxis] * small_scale_fading

  epsilon = 1e-10
  d_rx0 = calculate_distance(tx,rx[0])
  distances_rx0 = np.maximum(d_rx0, epsilon)

  d_rx1 = calculate_distance(tx,rx[1])
  distances_rx1 = np.maximum(d_rx0, epsilon)

  total_large_scale_fading_rx1 = cmath.sqrt(p_0/(distances_rx0**(3)))
  total_large_scale_fading_rx2 = cmath.sqrt(p_0/(distances_rx0**(3)))

  # Stack the large-scale fading values into an array
  total_large_scale_fading = np.array([total_large_scale_fading_rx1, total_large_scale_fading_rx2])

  # Generate small-scale fading
  small_scale_fading = np.random.rayleigh(scale=1.0, size=(1, len(rx), chan_realisations))

  # Multiply total large scale fading with small scale fading
  h = small_scale_fading * total_large_scale_fading.reshape(1,2,1)

  return h

  # for i in range(len(rx)):
  #   total_large_scale_fading_rx = np.sqrt(p_0/(calculate_distance(tx,rx[i])**(3)))
  #   for j in range(chan_realisations):
  #     small_scale_fading = np.random.rayleigh()
  #     h[:, i, j] =  total_large_scale_fading*small_scale_fading
  # return h

In [35]:
def generate_all_channels(p_0, tx_coords, rx_coords):
  """Generate channel realizations for all transmitter-receiver pairs.

    Args:
        p_0: reference path loss.
        tx_coords: list of 3D coordinates for transmitters [(x1, y1, z1), (x2, y2, z2)].
        rx_coords: list of 3D coordinates for receivers [(x1, y1, z1), (x2, y2, z2)].

    Returns:
        h_11, h_12, h_21, h_22: Channel model across all tx-rx pairs (shape: 1 x len(rx) x chan_realisations) e.g. h_21 means channel b/w transmitter 2 and receiver 1
    """
  h1 = generate_channel_model(p_0, tx_coords[0], rx_coords)
  h2 = generate_channel_model(p_0, tx_coords[1], rx_coords)

  # print(h1)
  # print(h1[:,0,:])
  h_11 = h1[:,0,:]
  h_12 = h1[:,1,:]

  h_21 = h2[:,0,:]
  h_22 = h2[:,1,:]

  return np.array([h_11, h_12, h_21, h_22])


In [45]:
# test
channel = generate_channel_model(30, [2,2,2], [[2,3,4],[4,3,2]])
print(channel)
print(channel.shape)

channels = generate_all_channels(-30, [[2,2,2],[1,1,1]], [[2,3,4],[4,3,2]])
print(channels.shape)


1.63807251762544
()
[[[1.75650429 2.30617697 1.89475218 2.66850038 1.41147246 2.08743478
   1.42075922 3.36013904 3.93542361 3.92007956 1.50970881 1.2803534
   1.68625483 1.82624731 1.51144117 2.19665866 1.48925706 2.29793786
   1.23610889 0.51810956 0.73612561 3.53183402 2.02014651 1.11845449
   1.26583604 2.36579744 0.17518899 1.28078942 3.50356934 1.49188456
   2.52010296 3.80119817 2.10044188 2.72658171 3.50163939 2.35724768
   3.60156627 1.45748494 2.82980418 0.4991723  0.76556118 2.64498933
   2.14993247 1.53271813 1.39606368 1.43599959 2.99864421 3.77681517
   2.21993609 1.88771839 0.36769396 3.57370805 3.75562655 2.37530606
   1.11364507 2.30947225 1.6522063  2.21250383 3.35072988 3.6387445
   0.93181849 5.73198624 0.47196551 2.12226532 2.26309712 2.15993043
   1.77330877 2.12753883 1.41287563 2.03861699 2.09019564 1.26304348
   2.41189211 1.28272299 0.84383334 3.00955676 3.30558021 3.06879796
   2.68959827 1.34557111 1.05249395 1.05086974 2.43420044 0.68949452
   0.13332151 2.

<ipython-input-44-a17d1c22e1e1>:36: RuntimeWarning: invalid value encountered in sqrt
  total_large_scale_fading_rx1 = np.sqrt(p_0/(calculate_distance(tx,rx[0])**(3)))
<ipython-input-44-a17d1c22e1e1>:37: RuntimeWarning: invalid value encountered in sqrt
  print(np.sqrt(p_0/(calculate_distance(tx,rx[0])**3)))
<ipython-input-44-a17d1c22e1e1>:39: RuntimeWarning: invalid value encountered in sqrt
  total_large_scale_fading_rx2 = np.sqrt(p_0/(calculate_distance(tx,rx[1])**(3)))


Step 3: Define channel gain

In [8]:
def generate_channel_gains(channels):
  return np.abs(channels)**2

In [48]:
# test
gains = generate_channel_gains(channels)
print(gains.shape)
print(gains)

(4, 1, 100)
[[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]]

 [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]]

 [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
   n

Step 4: Define spectral and energy efficiency functions

In [10]:
def spectral_efficiency(P, h, N0, W):
    """
    Calculate the total spectral efficiency for two transmitters.
    Parameters:
    - P: Array of transmit powers [P1, P2]
    - h: Array of channel gains [h11, h12, h21, h22] (first number is index of tx, second is index of rx)
    - N0: Noise power density
    - W: Bandwidth

    Returns:
    - Total spectral efficiency
    """
    P1, P2 = P
    # h11, h12, h21, h22 = h
    h11, h12, h21, h22 = h

    SE1 = np.log2(1 + h11 * P1 / (N0 * W + h21 * P2))
    SE2 = np.log2(1 + h22 * P2 / (N0 * W + h12 * P1))

    total_SE = SE1 + SE2

    return total_SE

In [13]:
def energy_efficiency(P, h, N0, W, Pc):
    """
    Calculate the energy efficiency for two transmitters.

    Parameters:
    - P: Array of transmit powers [P1, P2]
    - h: Array of channel gains [h11, h12, h21, h22]
    - N0: Noise power density
    - W: Bandwidth
    - Pc: Constant circuit power of the transmitter

    Returns:
    - Energy efficiency (EE)
    """
    h11, h12, h21, h22 = h
    P1, P2 = P

    # Calculate the spectral efficiencies for the given power configuration
    SE1 = np.log2(1 + h11 * P1 / (N0 * W + h21 * P2))
    SE2 = np.log2(1 + h22 * P2 / (N0 * W + h12 * P1))

    P1, P2 = P

    EE = SE1/(P1+Pc) + SE2/(P2+Pc)

    return EE

In [14]:
# test
se = spectral_efficiency([1,1], gains[:,:,9], -30, 10)
ee = energy_efficiency([1,1], gains[:,:,9], -30, 10, circuit_power)
ee

array([-0.03023552])

Step 5: Brute force search for optimal transmission powers

In [47]:
def find_optimal_power_on_se(h, N0, W):
    """
    Find the optimal transmit powers for two transmitters using binary power control.

    Parameters:
    - h: Array of channel gains [h11, h12, h21, h22]
    - N0: Noise power density
    - W: Bandwidth

    Returns:
    - Optimal power configuration (P1, P2)
    - Corresponding spectral efficiency
    """
    # Define the possible power configurations: [P1, P2]
    power_configurations = [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]

    # Initialize variables to store the best configuration and spectral efficiency
    best_power_configuration = None
    best_spectral_efficiency = -float('inf')
    best_channel_gain = None

    h_current = h[:, 0, :]  # Shape (2, 1, num_realizations)

    # Loop over all channel realizations
    for realization in range(h.shape[2]):
      # Iterate over all possible power configurations
      for P in power_configurations:
        se = spectral_efficiency(P, h_current[:, realization], N0, W)

        if se > best_spectral_efficiency:
          best_spectral_efficiency = se
          best_power_configuration = P
          best_channel_gain = h_current[:, realization]


    return best_channel_gain, best_power_configuration

In [16]:
# test
optimal_realisation, optimal_power = find_optimal_power_on_se(gains, noise_power_density, W)
print(f"Optimal Power Configuration: {optimal_power}")
print(f"Optimal Realisation: {optimal_realisation}")


Optimal Power Configuration: [1, 0]
Optimal Realisation: [32.22567373  0.58037383  0.2869169   0.37848834]


In [59]:
def find_optimal_power_on_ee(h, N0, W, Pc, threshold):
    """
    Find the optimal transmit powers for two transmitters to maximize energy efficiency.

    Parameters:
    - h: Array of channel gains [h11, h12, h21, h22]
    - N0: Noise power density
    - W: Bandwidth
    - Pc: Constant circuit power of the transmitter
    - threshold: Maximum transmit power for each transmitter

    Returns:
    - Optimal power configuration (P1, P2)
    - Corresponding energy efficiency
    """
    # Define the initial power configuration as [0, 0]
    initial_power = [0, 0]

    # Define bounds for each power: (0, max_threshold)
    bounds = [(0.0001, threshold), (0.0001, threshold)]

    # Initialize variables to store the best configuration and energy efficiency
    best_power_configuration = None
    best_energy_efficiency = -float('inf')
    best_channel_gain = None

    h_current = h[:, 0, :]  # Shape (2, 1, num_realizations)

    # Loop over all channel realizations
    for realization in range(h.shape[2]):
      # Extract channel gains for this realization
        h_curr = h_current[:, realization]

        # Objective function to minimize (negative of energy efficiency)
        def objective(P, h=h_curr):  # Capture h_current
            return -energy_efficiency(P, h, N0, W, Pc)

        # Perform optimization
        result = minimize(objective, initial_power, bounds=bounds, method='L-BFGS-B')

        # Optimal power configuration and corresponding energy efficiency
        optimal_power = result.x
        max_energy_efficiency = -result.fun  # Convert back to positive since we minimized the negative EE

        # Update best results if this realization is better
        if max_energy_efficiency > best_energy_efficiency:
            best_energy_efficiency = max_energy_efficiency
            best_power_configuration = optimal_power
            best_channel_gain = h_curr  # Store the corresponding channel gains

    return best_channel_gain, best_power_configuration

In [18]:
#test
opt_chann, optimal_power = find_optimal_power_on_ee(gains, noise_power_density, W, circuit_power, max_power)
print(f"Optimal Power Configuration: {optimal_power}")
print(f"Optimal Realisation: {opt_chann}")


Optimal Power Configuration: [0.07408823 0.0001    ]
Optimal Realisation: [ 8.4308651  12.4996966   0.38367694  1.02612121]


Generation of data

In [63]:
def generate_datasets(N0, W, Pc, threshold, filename_se, filename_ee):
  """
  Generate datasets for spectral efficiency and energy efficiency and save to text files.

  Parameters:
  - h: Array of channel gains (4x1x10)
  - N0: Noise power density
  - W: Bandwidth
  - Pc: Constant circuit power of the transmitter
  - max_threshold: Maximum transmit power for each transmitter
  - filename_se: Filename for spectral efficiency dataset
  - filename_ee: Filename for energy efficiency dataset
  """
  results_se = []
  results_ee = []

  normalized_channel_gains_se = np.zeros((num_samples,4))
  normalized_channel_gains_ee = np.zeros((num_samples, 4))
  normalized_tx_powers_se = np.zeros((num_samples, 2), dtype=int)
  normalized_tx_powers_ee = np.zeros((num_samples, 2), dtype=int)

  # with open(filename_se, 'w') as file_se, open(filename_ee, 'w') as file_ee:

  for sample in range(num_samples):

    # Generate coordinates for 2 transmitters and 2 receivers
    transmitters = generate_random_coordinates(2, x_range, y_range, z_range)
    receivers = generate_random_coordinates(2, x_range, y_range, z_range)

    # create channels b/w tx and rx
    channels = generate_all_channels(p_0, transmitters, receivers)

    # generate channel gains
    h = generate_channel_gains(channels)

    # Calculate optimal power for spectral efficiency
    channel_gains_se, tx_powers_se = find_optimal_power_on_se(h, N0, W)

    # Calculate optimal power for energy efficiency
    channel_gains_ee, tx_powers_ee = find_optimal_power_on_ee(h, N0, W, Pc, threshold)

    # Normalize channel gains and multiply by 100, then round to nearest integer
    mean_gain_se = np.mean(channel_gains_se)
    mean_gain_ee = np.mean(channel_gains_ee)

    std_dev_gain_se = np.std(channel_gains_se)
    std_dev_gain_ee = np.std(channel_gains_ee)

    if std_dev_gain_se > 0:
        normalized_channel_gains_se[sample] = np.round(((channel_gains_se - mean_gain_se) / std_dev_gain_se)*100).astype(int)
    else:
      normalized_channel_gains_se[sample]  = np.zeros_like(channel_gains_se)  # Avoid division by zero
    if std_dev_gain_ee > 0:
        normalized_channel_gains_ee[sample]  = np.round(((channel_gains_ee - mean_gain_ee) / std_dev_gain_ee)*100).astype(int)
    else:
      normalized_channel_gains_ee[sample]  = np.zeros_like(channel_gains_ee)

    # Normalize transmit powers and multiply by 100, then round to nearest integer
    normalized_tx_powers_se[sample]  = (np.array(tx_powers_se) / threshold) * 100
    normalized_tx_powers_ee[sample]  = (np.array(tx_powers_ee) / threshold) * 100
    normalized_tx_powers_se[sample]  = np.round(normalized_tx_powers_se[sample]).astype(int)
    normalized_tx_powers_ee[sample]  = np.round(normalized_tx_powers_ee[sample]).astype(int)

  # Prepare output strings in bulk
  for i in range(num_samples):
      line_se = f"If A is {', '.join(map(str, normalized_channel_gains_se[i]))}, then B is {normalized_tx_powers_se[i][0]}, {normalized_tx_powers_se[i][1]}.\n"
      results_se.append(line_se)

      line_ee = f"If A is {', '.join(map(str, normalized_channel_gains_ee[i]))}, then B is {normalized_tx_powers_ee[i][0]}, {normalized_tx_powers_ee[i][1]}.\n"
      results_ee.append(line_ee)

  # Write to files in one go
  with open(filename_se, 'w') as file_se:
      file_se.writelines(results_se)

  with open(filename_ee, 'w') as file_ee:
      file_ee.writelines(results_ee)

      # Write to spectral efficiency dataset
      # line_se = f"If A is {', '.join(map(str, normalized_channel_gains_se))}, then B is {normalized_tx_powers_se[0]}, {normalized_tx_powers_se[1]}.\n"
      # file_se.write(line_se)

      # # Write to energy efficiency dataset
      # line_ee = f"If A is {', '.join(map(str, normalized_channel_gains_ee))}, then B is {normalized_tx_powers_ee[0]}, {normalized_tx_powers_ee[1]}.\n"
      # file_ee.write(line_ee)

In [75]:
se_dataset = "se.txt"
ee_dataset = "ee.txt"
generate_datasets(noise_power_density, W, circuit_power, max_power, se_dataset, ee_dataset)